In [2]:
# used for data manipulation
import pandas as pd  

# used for working with csv files
import csv           

# We will display plots right inside Jupyter Notebook
%matplotlib inline
import matplotlib.pyplot as plt
# import os
# import conda

# conda_file_dir = conda.__file__
# conda_dir = conda_file_dir.split('lib')[0]
# proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
# os.environ["PROJ_LIB"] = proj_lib

#from mpl_toolkits.basemap import Basemap
# Map visualization
from mpl_toolkits.basemap import Basemap

# Heat Map
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

import numpy as np

# colour map
import matplotlib.cm

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'

# Increase the default plot size
from pylab import rcParams
rcParams['figure.figsize'] = 5, 4


# Disable warnings in Anaconda
import warnings
warnings.simplefilter('ignore')


In [3]:
data_with_latlon = pd.read_csv('UK_gpg_data.csv', encoding='utf-8')

In [4]:

# DiffMeanHourlyPercent       Their mean gender pay gap
# DiffMedianHourlyPercent     Their median gender pay gap
# DiffMeanBonusPercent        Their mean bonus gender pay gap
# DiffMedianBonusPercent      Their median bonus gender pay gap
# MaleBonusPercent            The proportion of men in the organisation receiving a bonus payment
# FemaleBonusPercent          The proportion of women the organisation receiving a bonus payment
# MaleLowerQuartile, FemaleLowerQuartile                 
# MaleLowerMiddleQuartile, FemaleLowerMiddleQuartile           
# MaleUpperMiddleQuartile, FemaleUpperMiddleQuartile           
# MaleTopQuartile, FemaleTopQuartile                
#                                      The proportion of men and women in each quartile pay band

In [5]:
# fig, ax = plt.subplots(figsize=(10,20))

# m = Basemap(resolution='h', # c, l, i, h, f or None
#             projection='merc',
#             lat_0=54.5, lon_0=-4.36,        #lat and lon of the centre point of your map
#             llcrnrlon=-6., llcrnrlat= 49.5, urcrnrlon=2., urcrnrlat=55.2) #lat and lon of your bounding box corners

# #set the colour of the seas and oceans on our map
# m.drawmapboundary(fill_color='#46bcec')

# #colour of land masses
# m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')

# #draws lines around the land masses
# m.drawcoastlines()


# def plot_area(pos):
#     count = pos['DiffMedianHourlyPercent']
#     x, y = m(pos['Longitude'], pos['Latitude'])
#     c = 'black'
#     l = 'l'
#     if pos['PaidMore'] == 'Men':
#         c = 'red'
#         l = 'Men Paid More'
#     elif pos['PaidMore'] == 'Women':
#         c = 'green'
#         l = 'Women Paid More'
#     else:
#         return
#     m.plot(x, y, 'o',color = c, markersize=3, alpha=0.6)
    
# m.plot([],[], 'o',color = 'red', label = 'Men Paid More')
# m.plot([],[], 'o',color = 'green', label = 'Women Paid More')
    
# data_with_latlon.apply(plot_area, axis=1)

# #draw postcode boundaries
# m.readshapefile('Areas', 'areas')

# plt.legend()


In [6]:
#extract area from postcodes
data_with_latlon['area'] = data_with_latlon['Postcode'].str[:2]

In [7]:
data_with_latlon.to_csv('withArea.csv', index=False)

In [8]:
data_try = data_with_latlon.copy()


In [9]:
area_PaidMore = data_try.groupby(['area','Target']).size().reset_index(name='count')

In [10]:
# Get count of women paid more in every area
area_Women = area_PaidMore[area_PaidMore['Target'] == 'Women']

In [11]:

def get_by_area(area, df):
    result = df[(data_with_latlon.Postcode.str.match('{}[^a-zA-Z].+'.format(area))) & (~data_with_latlon.Latitude.isnull()) & (~data_with_latlon.Longitude.isnull())]
    return result

def get_mean_area_lat_lon(area, df):
    area_houses = get_by_area(area, df) 
    mean_lat_lon = area_houses.Latitude.mean(), area_houses.Longitude.mean()
    return mean_lat_lon




In [12]:
# Get a mean latitude and longitude for each area
area_Women['pos'] = area_Women.area.apply(get_mean_area_lat_lon, args=(data_with_latlon,))

In [13]:
area_Women

,area,Target,count,pos
2,AB,Women,9,"(57.19927557142856, -2.158704519480519)"
5,AL,Women,9,"(51.77541649230766, -0.25515058461538453)"
8,B1,Women,2,"(52.47196087804879, -1.9048157317073169)"
11,B2,Women,1,"(52.484774, -1.82908055)"
14,B3,Women,3,"(52.46277380597017, -1.8449907313432832)"
17,B4,Women,4,"(52.45153140350879, -1.8525874385964922)"
22,B6,Women,7,"(52.45370736363636, -1.9977531363636365)"
25,B7,Women,5,"(52.552117386363626, -1.835614022727273)"
29,B9,Women,2,"(52.35958584615385, -1.8517387692307683)"
32,BA,Women,2,"(51.26783973846153, -2.4141097999999994)"


In [ ]:
#Ref: http://www.datadependence.com/2016/06/creating-map-visualisations-in-python/


fig, ax = plt.subplots(figsize=(10,20))
m = Basemap(resolution='h', # c, l, i, h, f or None
            projection='merc', 
            lat_0=54.5, lon_0=-4.36,
            llcrnrlon=-6., llcrnrlat= 49.5,urcrnrlon=2., urcrnrlat=55.2)
             
#set the colour of the seas and oceans on our map
m.drawmapboundary(fill_color='#46bcec')

#colour of land masses
m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')

#draws lines around the land masses
m.drawcoastlines()

#shapefile for regions
m.readshapefile('Areas', 'areas')


df_poly = pd.DataFrame({
        'shapes': [Polygon(np.array(shape), True) for shape in m.areas],
        'area': [area['name'] for area in m.areas_info]
    })
df_poly = df_poly.merge(area_Women, on='area', how='left')

#add colormap
cmap = plt.get_cmap('Greens')   
pc = PatchCollection(df_poly.shapes, zorder=2)
norm = Normalize()

pc.set_facecolor(cmap(norm(df_poly['count'].fillna(0).values)))
ax.add_collection(pc)

#add a colorbar
mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)

mapper.set_array(df_poly['count'])
plt.colorbar(mapper, shrink=0.4)


#Ref: http://introtopython.org/visualization_earthquakes.html#Making-a-simple-map

lons = [-0.126, -2.244644, -1.9, -2.978, -1.548, -1.15,  -1.132, -1.25596, 
        -0.9781, -1.6178, -1.5197, -2.7553, -2.3590,  -3.9436, -0.1218, -4.1427]
lats = [51.509,  53.483959, 52.481, 53.411, 53.796, 52.954, 
        52.639, 51.75222, 51.4543, 54.9783, 52.4068, 52.7073, 51.3811,  51.6214, 52.2053, 50.3755]
x,y = m(lons, lats)

x_offsets = [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]
y_offsets = [5000, 5000, -5000, -19000, 5000, 5000, 5000, 5000, -10000, 5000, -10000, -8000, 5000, 5000, 5000, 5000]


m.plot(x, y, '*', markersize=7)
 
labels = ['London', 'Manchester', 'Birmingham', 'Liverpool', 'Leeds', 'Nottingham', 'Leicester', 
          'Oxford', 'Reading', 'Newcastle upon Twyne', 'Coventry', 'Shrewsbury', 'Bath',  'Swansea', 'Cambridge', 'Plymouth']
for label, xpt, ypt, x_offset, y_offset in zip(labels, x, y, x_offsets, y_offsets):
    plt.text(xpt+x_offset, ypt+y_offset, label)

plt.title('Areas in UK where women are paid more')
    


plt.savefig('women_paid_more_map.svg')

In [ ]:
area_Neutral = area_PaidMore[area_PaidMore['Target'] == 'EqualPay']

In [ ]:
area_Neutral['pos'] = area_Neutral.area.apply(get_mean_area_lat_lon, args=(data_with_latlon,))

In [ ]:
#Ref: http://www.datadependence.com/2016/06/creating-map-visualisations-in-python/



fig, ax = plt.subplots(figsize=(10,20))
m = Basemap(resolution='h', # c, l, i, h, f or None
            projection='merc', 
            lat_0=54.5, lon_0=-4.36,
            llcrnrlon=-6., llcrnrlat= 49.5,urcrnrlon=2., urcrnrlat=55.2)
             
#set the colour of the seas and oceans on our map
m.drawmapboundary(fill_color='#46bcec')

#colour of land masses
m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')

#draws lines around the land masses
m.drawcoastlines()

#shapefile for regions
m.readshapefile('Areas', 'areas')


df_poly = pd.DataFrame({
        'shapes': [Polygon(np.array(shape), True) for shape in m.areas],
        'area': [area['name'] for area in m.areas_info]
    })
df_poly = df_poly.merge(area_Neutral, on='area', how='left')

#add colormap
cmap = plt.get_cmap('Reds')   
pc = PatchCollection(df_poly.shapes, zorder=2)
norm = Normalize()

pc.set_facecolor(cmap(norm(df_poly['count'].fillna(0).values)))
ax.add_collection(pc)

#add a colorbar
mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)

mapper.set_array(df_poly['count'])
plt.colorbar(mapper, shrink=0.4)


#Ref: http://introtopython.org/visualization_earthquakes.html#Making-a-simple-map

lons = [-0.126, -2.244644, -1.9, -2.978, -1.548, -1.15,  -1.132, -1.25596, 
        -0.9781, -1.6178, -1.5197, -2.7553, -2.3590,  -3.9436, -0.1218, -4.1427]
lats = [51.509,  53.483959, 52.481, 53.411, 53.796, 52.954, 
        52.639, 51.75222, 51.4543, 54.9783, 52.4068, 52.7073, 51.3811,  51.6214, 52.2053, 50.3755]
x,y = m(lons, lats)

x_offsets = [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]
y_offsets = [5000, 5000, -5000, -19000, 5000, 5000, 5000, 5000, -10000, 5000, -10000, -8000, 5000, 5000, 5000, 5000]


m.plot(x, y, '*', markersize=7)
 
labels = ['London', 'Manchester', 'Birmingham', 'Liverpool', 'Leeds', 'Nottingham', 'Leicester', 
          'Oxford', 'Reading', 'Newcastle upon Twyne', 'Coventry', 'Shrewsbury', 'Bath',  'Swansea', 'Cambridge', 'Plymouth']
for label, xpt, ypt, x_offset, y_offset in zip(labels, x, y, x_offsets, y_offsets):
    plt.text(xpt+x_offset, ypt+y_offset, label)

plt.title('Areas in UK for Equal Pay')
    


plt.savefig('equal_pay_map.svg')